In [23]:
import os
import numpy as np 
import pandas as pd
import nltk
import string
from nltk.stem.porter import PorterStemmer
import re
nltk.download('punkt')
nltk.download('stopwords')
stemmer = PorterStemmer()
stopwords = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to /home/nemesis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nemesis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# load data
df = pd.read_csv('questions.csv.zip')
df.dropna(how="any").reset_index(drop=True)

# train validate test split 70:20:10
from sklearn.model_selection import train_test_split
X_train_q1, X_test_q1, X_train_q2, X_test_q2, y_train, y_test = train_test_split(df['question1'], df['question2'], df['is_duplicate'], test_size=0.3, random_state=42, stratify=df['is_duplicate'])
X_val_q1, X_test_q1, X_val_q2, X_test_q2, y_val, y_test = train_test_split(X_test_q1, X_test_q2, y_test, test_size=(1/3), random_state=42, stratify=y_test)


Ratio of duplicate questions in the splits
Train set:  0.36919749967314835
Validation set:  0.36920279997031835
Test set:  0.369190432610255


In [6]:
y_train, y_val, y_test = np.array(y_train), np.array(y_val), np.array(y_test)
# ratio of duplicate questions in train, validation and test set
print("Ratio of duplicate questions in the splits")
print("Train set: ", y_train.sum()/len(y_train))
print("Validation set: ", y_val.sum()/len(y_val))
print("Test set: ", y_test.sum()/len(y_test))

Ratio of duplicate questions in the splits
Train set:  0.36919749967314835
Validation set:  0.36920279997031835
Test set:  0.369190432610255


In [19]:
INPUT = './input/'
TRAIN_LINEAR_PATH = INPUT + 'train_linear.csv.zip'
TEST_LINEAR_PATH = INPUT + 'test_linear.csv.zip'
VAL_LINEAR_PATH = INPUT + 'val_linear.csv.zip'

In [ ]:
train_linear = pd.DataFrame({'question1': X_train_q1, 'question2': X_train_q2, 'is_duplicate': y_train})
val_linear = pd.DataFrame({'question1': X_val_q1, 'question2': X_val_q2, 'is_duplicate': y_val})
test_linear = pd.DataFrame({'question1': X_test_q1, 'question2': X_test_q2, 'is_duplicate': y_test})
allQuestions = pd.concat((train_linear['question1'], train_linear['question2'])).reset_index(drop=True).astype(str)

In [22]:
# dump split files
os.makedirs(INPUT, exist_ok=True)  
train_linear.to_csv(TRAIN_LINEAR_PATH, index=False, compression='zip')
val_linear.to_csv(VAL_LINEAR_PATH, index=False, compression='zip')
test_linear.to_csv(TEST_LINEAR_PATH, index=False, compression='zip')

In [42]:
train_linear = pd.read_csv(TRAIN_LINEAR_PATH)
val_linear = pd.read_csv(VAL_LINEAR_PATH)
test_linear = pd.read_csv(TEST_LINEAR_PATH)
X_train_q1, X_train_q2, y_train = train_linear['question1'].astype('U').values, train_linear['question2'].astype('U').values, train_linear['is_duplicate'].values
X_val_q1, X_val_q2, y_val = val_linear['question1'].astype('U').values, val_linear['question2'].astype('U').values, val_linear['is_duplicate'].values
X_test_q1, X_test_q2, y_test = test_linear['question1'].astype('U').values, test_linear['question2'].astype('U').values, test_linear['is_duplicate'].values

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack as sparse_hstack, vstack as sparse_vstack, save_npz, load_npz

In [32]:
stemmer = PorterStemmer()

def tokenize(text: str) -> list[str]:
    tokens = nltk.word_tokenize(re.sub(r'[^\x00-\x7F]+',' ', text))
    tokens = [stemmer.stem(w) for w in tokens if stemmer.stem(w) not in stopwords]
    return tokens


In [1]:
N_GRAMS_PATH = './n_gram_features/'
UNIGRAM_PATH = N_GRAMS_PATH + 'unigrams_linear/'
BIGRAM_PATH = N_GRAMS_PATH + 'bigrams_linear/'
TRIGRAM_PATH = N_GRAMS_PATH + 'trigrams_linear/'
os.makedirs(N_GRAMS_PATH, exist_ok=True)
os.makedirs(UNIGRAM_PATH, exist_ok=True)
os.makedirs(BIGRAM_PATH, exist_ok=True)
os.makedirs(TRIGRAM_PATH, exist_ok=True)

### Creating Unigram features

In [44]:
unigramVectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,1), 
                        lowercase=True,
                        tokenizer=tokenize
                    )
                    
unigramVectorizer.fit(allQuestions)
q1_train = unigramVectorizer.transform(train_linear['question1'].astype(str))
q2_train = unigramVectorizer.transform(train_linear['question2'].astype(str))
X_train_unigram = sparse_hstack([q1_train, q2_train])
q1_val = unigramVectorizer.transform(val_linear['question1'].astype(str))
q2_val = unigramVectorizer.transform(val_linear['question2'].astype(str))
X_val_unigram = sparse_hstack([q1_val, q2_val])
q1_test = unigramVectorizer.transform(test_linear['question1'].astype(str))
q2_test = unigramVectorizer.transform(test_linear['question2'].astype(str))
X_test_unigram = sparse_hstack([q1_test, q2_test])

save_npz(UNIGRAM_PATH + "train.npz", X_train_unigram)
save_npz(UNIGRAM_PATH + "val.npz", X_val_unigram)
save_npz(UNIGRAM_PATH + "test.npz", X_test_unigram)

### Creating Bigram features

In [45]:
bigramVectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,2), 
                        lowercase=True,
                        tokenizer=tokenize
                    )
                    
bigramVectorizer.fit(allQuestions)
q1_train = bigramVectorizer.transform(train_linear['question1'].astype(str))
q2_train = bigramVectorizer.transform(train_linear['question2'].astype(str))
X_train_bigram = sparse_hstack([q1_train, q2_train])
q1_val = bigramVectorizer.transform(val_linear['question1'].astype(str))
q2_val = bigramVectorizer.transform(val_linear['question2'].astype(str))
X_val_bigram = sparse_hstack([q1_val, q2_val])
q1_test = bigramVectorizer.transform(test_linear['question1'].astype(str))
q2_test = bigramVectorizer.transform(test_linear['question2'].astype(str))
X_test_bigram = sparse_hstack([q1_test, q2_test])

save_npz(BIGRAM_PATH + "train.npz", X_train_bigram)
save_npz(BIGRAM_PATH + "val.npz", X_val_bigram)
save_npz(BIGRAM_PATH + "test.npz", X_test_bigram)

### Creating Trigram features

In [46]:
trigramVectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,3), 
                        lowercase=True,
                        tokenizer=tokenize
                    )
                    
trigramVectorizer.fit(allQuestions)
q1_train = trigramVectorizer.transform(train_linear['question1'].astype(str))
q2_train = trigramVectorizer.transform(train_linear['question2'].astype(str))
X_train_trigram = sparse_hstack([q1_train, q2_train])
q1_val = trigramVectorizer.transform(val_linear['question1'].astype(str))
q2_val = trigramVectorizer.transform(val_linear['question2'].astype(str))
X_val_trigram = sparse_hstack([q1_val, q2_val])
q1_test = trigramVectorizer.transform(test_linear['question1'].astype(str))
q2_test = trigramVectorizer.transform(test_linear['question2'].astype(str))
X_test_trigram = sparse_hstack([q1_test, q2_test])

save_npz(TRIGRAM_PATH + "train.npz", X_train_trigram)
save_npz(TRIGRAM_PATH + "val.npz", X_val_trigram)
save_npz(TRIGRAM_PATH + "test.npz", X_test_trigram)

## Logistic Regression

### Unigrams

In [53]:
# X_train_unigram = load_npz(UNIGRAM_PATH + "train.npz")
# X_test_unigram = load_npz(UNIGRAM_PATH + "test.npz")
unigramLogisticRegressor = SGDClassifier(
                            loss='log_loss', 
                            penalty='l2', 
                            alpha=0.00001, 
                            max_iter=1000,
                            n_iter_no_change=20,
                            learning_rate='optimal',
                            n_jobs=-1,
                            random_state=42)
unigramLogisticRegressor.fit(X_train_unigram, y_train)
y_pred_unigram_logistic = unigramLogisticRegressor.predict(X_test_unigram)
print("Unigram Logistic Regression Accuracy: ", accuracy_score(y_test, y_pred_unigram_logistic))
print("Unigram Logistic Regression F1 Score: ", f1_score(y_test, y_pred_unigram_logistic))

Unigram Logistic Regression Accuracy:  0.7418437260382399
Unigram Logistic Regression F1 Score:  0.6310840903467534


### Bigrams

In [54]:
# X_train_bigram = load_npz(BIGRAM_PATH + "train.npz")
# X_test_bigram = load_npz(BIGRAM_PATH + "test.npz")
bigramLogisticRegressor = SGDClassifier(
                            loss='log_loss', 
                            penalty='l2', 
                            alpha=0.00001, 
                            max_iter=1000,
                            n_iter_no_change=20,
                            learning_rate='optimal',
                            n_jobs=-1,
                            random_state=42)
bigramLogisticRegressor.fit(X_train_bigram, y_train)
y_pred_bigram_logistic = bigramLogisticRegressor.predict(X_test_bigram)
print("bigram Logistic Regression Accuracy: ", accuracy_score(y_test, y_pred_bigram_logistic))
print("bigram Logistic Regression F1 Score: ", f1_score(y_test, y_pred_bigram_logistic))

bigram Logistic Regression Accuracy:  0.7962106408765984
bigram Logistic Regression F1 Score:  0.7066405554566494


### Trigrams

In [55]:
# X_train_trigram = load_npz(TRIGRAM_PATH + "train.npz")
# X_test_trigram = load_npz(TRIGRAM_PATH + "test.npz")
trigramLogisticRegressor = SGDClassifier(
                            loss='log_loss', 
                            penalty='l2', 
                            alpha=0.00001, 
                            max_iter=1000,
                            n_iter_no_change=20,
                            learning_rate='optimal',
                            n_jobs=-1,
                            random_state=42)
trigramLogisticRegressor.fit(X_train_trigram, y_train)
y_pred_trigram_logistic = trigramLogisticRegressor.predict(X_test_trigram)
print("trigram Logistic Regression Accuracy: ", accuracy_score(y_test, y_pred_trigram_logistic))
print("trigram Logistic Regression F1 Score: ", f1_score(y_test, y_pred_trigram_logistic))

trigram Logistic Regression Accuracy:  0.8114472284746098
trigram Logistic Regression F1 Score:  0.7147614593077642


### Trigrams Tuned

Applying GridSearchCV on Trigrams model to get the best set of parameters

In [81]:
trigramLogisticRegressor = SGDClassifier(
                            loss='log_loss',
                            penalty='l2',
                            max_iter=1000,
                            learning_rate='optimal',
                            n_jobs=-1,
                            random_state=42)
parameters = dict({
                'alpha':[0.01, 0.001, 0.0001, 0.00001, 0.000001],
                'n_iter_no_change': [5, 10, 15, 20]
            })
cv_stratifier_splitter = StratifiedKFold(n_splits=5)
grid_search = GridSearchCV(trigramLogisticRegressor, 
                            parameters, 
                            cv=cv_stratifier_splitter, 
                            scoring=['accuracy', 'f1'], 
                            n_jobs=-1,
                            refit='f1')
grid_search.fit(sparse_vstack([X_train_trigram, X_val_trigram]), np.concatenate((y_train, y_val)))


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SGDClassifier(loss='log_loss', n_jobs=-1,
                                     random_state=42),
             n_jobs=-1,
             param_grid={'alpha': [0.01, 0.001, 0.0001, 1e-05, 1e-06],
                         'n_iter_no_change': [5, 10, 15, 20]},
             refit='f1', scoring=['accuracy', 'f1'])

In [86]:
gridSearchCVResults = pd.DataFrame.from_dict(grid_search.cv_results_).reset_index()
gridSearchCVResults

,index,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_n_iter_no_change,params,split0_test_accuracy,split1_test_accuracy,...,std_test_accuracy,rank_test_accuracy,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0,32.057936,4.823794,1.462032,0.469907,0.01,5,"{'alpha': 0.01, 'n_iter_no_change': 5}",0.692152,0.691969,...,0.000602,17,0.352870,0.351276,0.352789,0.354524,0.356614,0.353614,0.001818,17
1,1,34.839753,5.847555,1.570614,0.270366,0.01,10,"{'alpha': 0.01, 'n_iter_no_change': 10}",0.691190,0.691969,...,0.000497,18,0.347076,0.351426,0.348466,0.349686,0.349993,0.349329,0.001469,18
2,2,41.940447,10.329488,1.597674,0.275955,0.01,15,"{'alpha': 0.01, 'n_iter_no_change': 15}",0.691122,0.691461,...,0.000457,19,0.346570,0.348641,0.347652,0.350843,0.350410,0.348823,0.001617,19
3,3,44.677039,6.685752,1.307405,0.393770,0.01,20,"{'alpha': 0.01, 'n_iter_no_change': 20}",0.690916,0.690843,...,0.000539,20,0.345848,0.345646,0.346039,0.347637,0.349897,0.347013,0.001605,20
4,4,24.846832,6.631401,1.670751,0.193314,0.001,5,"{'alpha': 0.001, 'n_iter_no_change': 5}",0.741415,0.742827,...,0.001528,14,0.546314,0.550779,0.547320,0.547302,0.555941,0.549531,0.003546,14
5,5,31.927570,2.169471,0.891322,0.097146,0.001,10,"{'alpha': 0.001, 'n_iter_no_change': 10}",0.742473,0.743638,...,0.001614,13,0.552838,0.556443,0.547028,0.549382,0.549654,0.551069,0.003262,13
6,6,35.785039,6.948313,1.889833,0.513881,0.001,15,"{'alpha': 0.001, 'n_iter_no_change': 15}",0.741072,0.743129,...,0.001499,15,0.545130,0.551458,0.546284,0.545874,0.550921,0.547933,0.002690,15
7,7,59.535236,2.297733,1.418594,0.290130,0.001,20,"{'alpha': 0.001, 'n_iter_no_change': 20}",0.741236,0.741796,...,0.001460,16,0.546055,0.544507,0.539016,0.542346,0.547395,0.543864,0.002949,16
8,8,32.525184,8.522953,1.530440,0.418499,0.0001,5,"{'alpha': 0.0001, 'n_iter_no_change': 5}",0.780660,0.782870,...,0.001323,9,0.652092,0.650675,0.650521,0.648484,0.657971,0.651949,0.003223,10
9,9,49.180857,11.419878,1.625803,0.334466,0.0001,10,"{'alpha': 0.0001, 'n_iter_no_change': 10}",0.780578,0.782719,...,0.001481,12,0.650713,0.653565,0.643136,0.640263,0.642371,0.646010,0.005172,12


<u>Best Parameters</u>

In [87]:
print("Best Parameters: ", grid_search.best_params_)

Best Parameters:  {'alpha': 1e-06, 'n_iter_no_change': 20}


In [92]:
bestAlpha = grid_search.best_params_['alpha']
bestNIterNoChange = grid_search.best_params_['n_iter_no_change']
trigramTunedLogisticRegressor = SGDClassifier(
                            loss='log_loss',
                            alpha=bestAlpha,
                            penalty='l2',
                            max_iter=1000,
                            learning_rate='optimal',
                            n_iter_no_change=bestNIterNoChange,
                            n_jobs=-1,
                            random_state=42)
trigramTunedLogisticRegressor.fit(X_train_trigram, y_train)
y_pred_trigram_tuned_logistic = trigramTunedLogisticRegressor.predict(X_test_trigram)
print("trigram Tuned Logistic Regression Accuracy: ", 100*accuracy_score(y_test, y_pred_trigram_tuned_logistic))
print("trigram Tuned Logistic Regression F1 Score: ", 100*f1_score(y_test, y_pred_trigram_tuned_logistic))


trigram Tuned Logistic Regression Accuracy:  80.31116277919315
trigram Tuned Logistic Regression F1 Score:  72.37454015409178


## SVM

According to paper, the default parameters to be used for SVM are $C=1.0$ and $kernel=linear$ unless specified otherwise.

##### Unigram Linear Model 

In [8]:
# X_train_unigram = load_npz(UNIGRAM_PATH + "train.npz")
# X_test_unigram = load_npz(UNIGRAM_PATH + "test.npz")
unigramLinearSVM = LinearSVC(C=1.0, max_iter=10000, random_state=42)
unigramLinearSVM.fit(X_train_unigram, y_train)
y_pred_unigram_linear_svm = unigramLinearSVM.predict(X_test_unigram)
print("Unigram Linear SVM Accuracy: ", accuracy_score(y_test, y_pred_unigram_linear_svm))
print("Unigram Linear SVM F1 Score: ", f1_score(y_test, y_pred_unigram_linear_svm))

Unigram Linear SVM Accuracy:  0.7339038808775878
Unigram Linear SVM F1 Score:  0.6413282656531306


##### Bigram Linear Model

In [8]:
# X_train_bigram = load_npz(BIGRAM_PATH + "train.npz")
# X_test_bigram = load_npz(BIGRAM_PATH + "test.npz")
bigramLinearSVM = LinearSVC(C=1.0, max_iter=10000, random_state=42)
bigramLinearSVM.fit(X_train_bigram, y_train)
y_pred_bigram_linear_svm = bigramLinearSVM.predict(X_test_bigram)
print("Bigram Linear SVM Accuracy: ", accuracy_score(y_test, y_pred_bigram_linear_svm))
print("Bigram Linear SVM F1 Score: ", f1_score(y_test, y_pred_bigram_linear_svm))

Bigram Linear SVM Accuracy:  0.7765465383759182
Bigram Linear SVM F1 Score:  0.6993877279382404


##### Trigram Linear Model

In [9]:
# X_train_trigram = load_npz(TRIGRAM_PATH + "train.npz")
# X_test_trigram = load_npz(TRIGRAM_PATH + "test.npz")
trigramLinearSVM = LinearSVC(C=0.1, max_iter=10000, random_state=42)
trigramLinearSVM.fit(X_train_trigram, y_train)
y_pred_trigram_linear_svm = trigramLinearSVM.predict(X_test_trigram)
print("Trigram Linear SVM Accuracy: ", accuracy_score(y_test, y_pred_trigram_linear_svm))
print("Trigram Linear SVM F1 Score: ", f1_score(y_test, y_pred_trigram_linear_svm))

Trigram Linear SVM Accuracy:  0.8075391426946004
Trigram Linear SVM F1 Score:  0.7242637938977285


### Parameter tuning for Trigram SVM Model with different kernels

In [8]:
# X_train_trigram = load_npz(TRIGRAM_PATH + "train.npz")
# X_test_trigram = load_npz(TRIGRAM_PATH + "test.npz")
# X_val_trigram = load_npz(TRIGRAM_PATH + "val.npz")

In [9]:
parameters = dict({
                    'C':[0.005, 0.001, 0.1, 0.5, 1.0, 10, 50], 
                    'kernel':['linear', 'rbf', 'poly', 'sigmoid']
                })
trigramSVM = SVC(max_iter=5000, random_state=42)
cv_stratified_splitter = StratifiedKFold(n_splits=2)
grid_search = GridSearchCV(trigramSVM, 
                            parameters, 
                            cv=cv_stratified_splitter, 
                            scoring=['accuracy', 'f1'], 
                            n_jobs=-1,
                            refit='accuracy',
                            verbose=1)
grid_search.fit(sparse_vstack([X_train_trigram, X_val_trigram]), np.concatenate((y_train, y_val)))

Fitting 2 folds for each of 28 candidates, totalling 56 fits


/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: 

GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
             estimator=SVC(max_iter=5000, random_state=42), n_jobs=-1,
             param_grid={'C': [0.005, 0.001, 0.1, 0.5, 1.0, 10, 50],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             refit='accuracy', scoring=['accuracy', 'f1'], verbose=1)

In [13]:
gridSearchCVResults = pd.DataFrame.from_dict(grid_search.cv_results_)
gridSearchCVResults

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_accuracy,split1_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_f1,split1_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,1269.840520,33.217703,496.621304,2.677389,0.005,linear,"{'C': 0.005, 'kernel': 'linear'}",0.372685,0.369268,0.370977,0.001708,22,0.540522,0.539318,0.539920,0.000602,11
1,1282.469714,30.366632,477.714923,7.902120,0.005,rbf,"{'C': 0.005, 'kernel': 'rbf'}",0.601019,0.515374,0.558197,0.042823,11,0.524347,0.550310,0.537329,0.012981,20
2,1258.852223,24.312050,482.105273,2.530365,0.005,poly,"{'C': 0.005, 'kernel': 'poly'}",0.369200,0.369197,0.369199,0.000002,28,0.539293,0.539290,0.539292,0.000002,17
3,1313.461902,23.086716,555.533464,7.870833,0.005,sigmoid,"{'C': 0.005, 'kernel': 'sigmoid'}",0.389230,0.371027,0.380129,0.009101,21,0.544587,0.539890,0.542238,0.002349,10
4,1264.621557,28.248824,490.529670,1.299316,0.001,linear,"{'C': 0.001, 'kernel': 'linear'}",0.370992,0.369785,0.370389,0.000604,23,0.539908,0.539492,0.539700,0.000208,12
5,1284.942240,30.085127,483.316809,7.461044,0.001,rbf,"{'C': 0.001, 'kernel': 'rbf'}",0.599777,0.514225,0.557001,0.042776,12,0.526669,0.549251,0.537960,0.011291,19
6,1137.083540,19.191614,515.709534,13.700713,0.001,poly,"{'C': 0.001, 'kernel': 'poly'}",0.369415,0.369197,0.369306,0.000109,26,0.539374,0.539290,0.539332,0.000042,15
7,1172.080629,26.647017,594.919960,3.235845,0.001,sigmoid,"{'C': 0.001, 'kernel': 'sigmoid'}",0.391110,0.382130,0.386620,0.004490,19,0.544888,0.542999,0.543944,0.000945,7
8,1119.065827,21.330025,578.916634,4.447443,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.498425,0.560903,0.529664,0.031239,14,0.544728,0.553033,0.548881,0.004153,4
9,1147.327423,23.515550,485.715543,0.260239,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.600821,0.527950,0.564386,0.036435,10,0.549873,0.546554,0.548213,0.001660,5


<u>Best Parameters</u>

In [14]:
print("Best Parameters: ", grid_search.best_params_)

Best Parameters:  {'C': 10, 'kernel': 'rbf'}


In [15]:
bestC=grid_search.best_params_['C']
bestKernel=grid_search.best_params_['kernel']
trigramTunedSVM = SVC(C=bestC, kernel=bestKernel, max_iter=5000, random_state=42)
trigramTunedSVM.fit(X_train_trigram, y_train)
y_pred_trigram_tuned_svm = trigramTunedSVM.predict(X_test_trigram)
print("Trigram Tuned SVM Accuracy: ", accuracy_score(y_test, y_pred_trigram_tuned_svm))
print("Trigram Tuned SVM F1 Score: ", f1_score(y_test, y_pred_trigram_tuned_svm))

/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Trigram Tuned SVM Accuracy:  0.6275445843330283
Trigram Tuned SVM F1 Score:  0.365177065767285


## Sentence embeddings as feature vectors

50-dimensional word vectors are obtained using GloVe vectors (GloVe.6B.50d.txt). The sentence embeddings are obtained by simply summing the word embeddings in a sentence. The sentence embeddings are then used as feature vectors for classification in the following two ways:
- Plain sentence embeddings
- Distance measure between vectors

In [85]:
GLOVE_PATH = INPUT + 'glove.6B.50d.txt'

In [79]:
def preprocessAndTokenizeForGlove(text: str) -> list[str]:
    text = re.sub(r'[^\x00-\x7F]+',' ', text.lower())
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(w) for w in tokens if stemmer.stem(w) not in stopwords]
    return tokens

preprocessorAndTokenizor = np.vectorize(preprocessAndTokenizeForGlove)

In [83]:
try:
    X_train_q1_tokenized = np.array([preprocessAndTokenizeForGlove(ques) for ques in X_train_q1], dtype=object)
    X_train_q2_tokenized = np.array([preprocessAndTokenizeForGlove(ques) for ques in X_train_q2], dtype=object)
    X_test_q1_tokenized = np.array([preprocessAndTokenizeForGlove(ques) for ques in X_test_q1], dtype=object)
    X_test_q2_tokenized = np.array([preprocessAndTokenizeForGlove(ques) for ques in X_test_q2], dtype=object)
    X_val_q1_tokenized = np.array([preprocessAndTokenizeForGlove(ques) for ques in X_val_q1], dtype=object)
    X_val_q2_tokenized = np.array([preprocessAndTokenizeForGlove(ques) for ques in X_val_q2], dtype=object)
except Exception as e:
    print(e)

In [86]:
# Load GloVe Word Embeddings
GloVe_embeddings = {}
with open(GLOVE_PATH, 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        GloVe_embeddings[word] = vector

### Creating sentence embeddings for each question

According to the paper, the sentence embeddings are obtained by simply summing the embeddings of all the tokens in a sentence.

In [181]:
X_train_q1_embeddings = np.array([np.sum([GloVe_embeddings[w] for w in ques if w in GloVe_embeddings] + [np.zeros((50,))], axis=0) for ques in X_train_q1_tokenized])
X_train_q2_embeddings = np.array([np.sum([GloVe_embeddings[w] for w in ques if w in GloVe_embeddings] + [np.zeros((50,))], axis=0) for ques in X_train_q2_tokenized])
X_test_q1_embeddings = np.array([np.sum([GloVe_embeddings[w] for w in ques if w in GloVe_embeddings] + [np.zeros((50,))], axis=0) for ques in X_test_q1_tokenized])
X_test_q2_embeddings = np.array([np.sum([GloVe_embeddings[w] for w in ques if w in GloVe_embeddings] + [np.zeros((50,))], axis=0) for ques in X_test_q2_tokenized])
X_val_q1_embeddings = np.array([np.sum([GloVe_embeddings[w] for w in ques if w in GloVe_embeddings] + [np.zeros((50,))], axis=0) for ques in X_val_q1_tokenized])
X_val_q2_embeddings = np.array([np.sum([GloVe_embeddings[w] for w in ques if w in GloVe_embeddings] + [np.zeros((50,))], axis=0) for ques in X_val_q2_tokenized])

In [182]:
QUESTION_EMBEDDINGS = './question_embeddings/'
os.makedirs(QUESTION_EMBEDDINGS, exist_ok=True)
TRAIN_Q1_EMBEDDINGS = QUESTION_EMBEDDINGS + 'train_q1_embeddings.npz'
TRAIN_Q2_EMBEDDINGS = QUESTION_EMBEDDINGS + 'train_q2_embeddings.npz'
TEST_Q1_EMBEDDINGS = QUESTION_EMBEDDINGS + 'test_q1_embeddings.npz'
TEST_Q2_EMBEDDINGS = QUESTION_EMBEDDINGS + 'test_q2_embeddings.npz'
VAL_Q1_EMBEDDINGS = QUESTION_EMBEDDINGS + 'val_q1_embeddings.npz'
VAL_Q2_EMBEDDINGS = QUESTION_EMBEDDINGS + 'val_q2_embeddings.npz'

In [183]:
np.savez_compressed(TRAIN_Q1_EMBEDDINGS, X_train_q1_embeddings)
np.savez_compressed(TRAIN_Q2_EMBEDDINGS, X_train_q2_embeddings)
np.savez_compressed(TEST_Q1_EMBEDDINGS, X_test_q1_embeddings)
np.savez_compressed(TEST_Q2_EMBEDDINGS, X_test_q2_embeddings)
np.savez_compressed(VAL_Q1_EMBEDDINGS, X_val_q1_embeddings)
np.savez_compressed(VAL_Q2_EMBEDDINGS, X_val_q2_embeddings)

In [ ]:
# X_train_q1_embeddings = np.load(TRAIN_Q1_EMBEDDINGS)['arr_0']
# X_train_q2_embeddings = np.load(TRAIN_Q2_EMBEDDINGS)['arr_0']
# X_test_q1_embeddings = np.load(TEST_Q1_EMBEDDINGS)['arr_0']
# X_test_q2_embeddings = np.load(TEST_Q2_EMBEDDINGS)['arr_0']
# X_val_q1_embeddings = np.load(VAL_Q1_EMBEDDINGS)['arr_0']
# X_val_q2_embeddings = np.load(VAL_Q2_EMBEDDINGS)['arr_0']

### 1. Plain sentence embeddings

$100$-dimensional feature vector = $50$-dimensional question $1$ sentence embedding + $50$-dimensional question $2$ sentence embedding

In [188]:
X_train_plain_embeddings = np.hstack((X_train_q1_embeddings, X_train_q2_embeddings))
X_test_plain_embeddings = np.hstack((X_test_q1_embeddings, X_test_q2_embeddings))
X_val_plain_embeddings = np.hstack((X_val_q1_embeddings, X_val_q2_embeddings))

In [191]:
SVMmodel = SVC(kernel='rbf', C=0.19, random_state=42, max_iter=3500)
SVMmodel.fit(X_train_plain_embeddings, y_train)
y_pred = SVMmodel.predict(X_test_plain_embeddings)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.4918499097182715
F1 Score: 0.5580414766371224


In [192]:
X_train_plain_embeddings.shape, X_test_plain_embeddings.shape, X_val_plain_embeddings.shape

((283003, 100), (40429, 100), (80858, 100))

In [193]:
SVMmodel = SVC(kernel='rbf', C=1.0, random_state=42, max_iter=3500)
SVMmodel.fit(X_train_plain_embeddings, y_train)
y_pred = SVMmodel.predict(X_test_plain_embeddings)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.48846125306092164
F1 Score: 0.5588053333333333


In [194]:
parameters = {
    'C': [0.005, 0.01, 0.1, 0.5, 1.0, 10, 50],
    'kernel': ['linear', 'rbf']
}
plainEmbeddingSVM = SVC(max_iter=3500, random_state=42)
cv_stratified_splitter = StratifiedKFold(n_splits=5)
plainEmbeddingSVMGridSearch = GridSearchCV(plainEmbeddingSVM, parameters, cv=5, n_jobs=-1, verbose=1)
plainEmbeddingSVMGridSearch.fit(X_train_plain_embeddings, y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: 

GridSearchCV(cv=5, estimator=SVC(max_iter=3500, random_state=42), n_jobs=-1,
             param_grid={'C': [0.005, 0.01, 0.1, 0.5, 1.0, 10, 50],
                         'kernel': ['linear', 'rbf']},
             verbose=1)

In [196]:
import pickle
with open('plainEmbeddingSVMGridSearch.pkl', 'wb') as f:
    pickle.dump(plainEmbeddingSVMGridSearch, f)
    f.close()

In [197]:
plainEmbeddingSVMGridSearch.best_params_

{'C': 50, 'kernel': 'rbf'}

In [198]:
plainEmbeddingSVMGridSearch.best_score_

0.6034883332793319

### 2. Distance measure between vectors
Feature vector is obtained by taking various distance measures between the sentence embeddings of the two questions:
- Bray Curtis distance:
$$
d_{Bray Curtis} = \frac{1}{2} \sum_{i=1}^{50} \frac{|q_1[i] - q_2[i]|}{|q_1[i] + q_2[i]|}
$$
- Canberra distance:
$$
d_{Canberra} = \sum_{i=1}^{50} \frac{|q_1[i] - q_2[i]|}{|q_1[i]| + |q_2[i]|}
$$
- Chebyshev distance:
$$d_{Chebyshev} = max(|q_1[i] - q_2[i]|)$$
- City block distance:
$$d_{City block} = \sum_{i=1}^{50} |q_1[i] - q_2[i]|$$
- Correlation distance:
$$
\begin{aligned}
d_{Correlation} = 1 - \frac{\sum_{i=1}^{50} (q_1[i] - \bar{q_1})(q_2[i] - \bar{q_2})}{\sqrt{\sum_{i=1}^{50} (q_1[i] - \bar{q_1})^2} \sqrt{\sum_{i=1}^{50} (q_2[i] - \bar{q_2})^2}}
\end{aligned}
$$
- Cosine distance: 
$$
d_{Cosine} = 1 - \frac{\sum_{i=1}^{50} q_1[i]q_2[i]}{\sqrt{\sum_{i=1}^{50} q_1[i]^2} \sqrt{\sum_{i=1}^{50} q_2[i]^2}}
$$
- Euclidean distance:
$$d_{Euclidean} = \sqrt{\sum_{i=1}^{50} (q_1[i] - q_2[i])^2}$$

In [201]:
import scipy.spatial.distance as scipyDistance

def distances(q1, q2):
    distanceFeatureVector = np.array([
        scipyDistance.braycurtis(q1, q2),
        scipyDistance.canberra(q1, q2),
        scipyDistance.chebyshev(q1, q2),
        scipyDistance.cityblock(q1, q2),
        scipyDistance.correlation(q1, q2),
        scipyDistance.cosine(q1, q2),
        scipyDistance.euclidean(q1, q2)
    ])
    distanceFeatureVector = np.nan_to_num(distanceFeatureVector)
    return distanceFeatureVector


X_train_distances = np.array([distances(q1, q2) for q1, q2 in zip(X_train_q1_embeddings, X_train_q2_embeddings)])
X_test_distances = np.array([distances(q1, q2) for q1, q2 in zip(X_test_q1_embeddings, X_test_q2_embeddings)])
X_val_distances = np.array([distances(q1, q2) for q1, q2 in zip(X_val_q1_embeddings, X_val_q2_embeddings)])

/home/nemesis/.local/lib/python3.10/site-packages/scipy/spatial/distance.py:1155: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()
/home/nemesis/.local/lib/python3.10/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [202]:
distanceSVM = SVC(kernel='rbf', C=50, random_state=42, max_iter=3500)
distanceSVM.fit(X_train_distances, y_train)
y_pred = distanceSVM.predict(X_test_distances)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.40960696529718765
F1 Score: 0.5550895636451753


In [203]:
distanceSVM = SVC(kernel='linear', C=1.0, random_state=42, max_iter=3500)
distanceSVM.fit(X_train_distances, y_train)
y_pred = distanceSVM.predict(X_test_distances)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.37020455613544734
F1 Score: 0.5396825396825397


In [204]:
distanceSVM = SVC(kernel='linear', C=10, random_state=42, max_iter=3500)
distanceSVM.fit(X_train_distances, y_train)
y_pred = distanceSVM.predict(X_test_distances)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.3794306067426847
F1 Score: 0.5432633667692195


In [205]:
distanceSVM = SVC(kernel='linear', C=50, random_state=42, max_iter=3500)
distanceSVM.fit(X_train_distances, y_train)
y_pred = distanceSVM.predict(X_test_distances)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

/home/nemesis/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=3500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.3794306067426847
F1 Score: 0.5432633667692195
